In [4]:
from Qlearning import QlearningAgent
import numpy as np
from tic_env import TictactoeEnv, OptimalPlayer
env = TictactoeEnv()

Turns = np.array(['X','O'])

agentWins = 0
player_optWins = 0

player_opt = OptimalPlayer(epsilon=0.5, player=Turns[0])
agent = QlearningAgent(epsilon=0.5, player=Turns[1])

for nbGames in range(100):
    env.reset()
    grid, _, __ = env.observe()
    Turns = Turns[np.random.permutation(2)]

    player_opt.player = Turns[0]
    agent.player = Turns[1]

    for roundGame in range(9):
        if env.current_player == player_opt.player:
            if (roundGame > 1) and isinstance(player_opt, QlearningAgent):
                player_opt.learn(grid, 0)
            move = player_opt.act(grid)
        else:
            if (roundGame > 1) and isinstance(agent, QlearningAgent):
                agent.learn(grid, 0)
            move = agent.act(grid)            

        grid, end, winner = env.step(move, print_grid=False)

        if end:
            if isinstance(player_opt, QlearningAgent):
                if winner == player_opt.player:
                    player_opt.learn(grid, 1, end=True)
                elif winner == agent.player:
                    player_opt.learn(grid, -1, end=True)
                else:
                    player_opt.learn(grid, 0, end=True)
            if isinstance(agent, QlearningAgent):
                if winner == agent.player:
                    agent.learn(grid, 1, end=True)
                elif winner == player_opt.player:
                    agent.learn(grid, -1, end=True)
                else:
                    agent.learn(grid, 0, end=True)
            print("-------------------------------------------")
            print(f"Game end, winner is player {winner}")
            print(f"Optimal player = {Turns[0]}")
            print(f"RL Agent player = {Turns[1]}")

            agentWins += int(winner == agent.player)
            player_optWins += int(winner == player_opt.player)

            env.render()
            env.reset()
            break
agentRatio = agentWins/nbGames
print(f"Agent wins {agentWins}/{nbGames} = {agentRatio}")
player_optRatio = player_optWins/nbGames
print(f"Optimal player wins {player_optWins}/{nbGames} = {player_optRatio}")
agentAverageReward = (agentWins-player_optWins)/nbGames
print(f"Agent average reward = {agentAverageReward}")
player_optAverageReward = (player_optWins-agentWins)/nbGames
print(f"Optimal player average reward = {player_optAverageReward}")

-------------------------------------------
Game end, winner is player X
Optimal player = X
RL Agent player = O
|X X X|
|- X O|
|- O O|

-------------------------------------------
Game end, winner is player O
Optimal player = O
RL Agent player = X
|O X -|
|X O -|
|X - O|

-------------------------------------------
Game end, winner is player X
Optimal player = O
RL Agent player = X
|- X O|
|- X -|
|- X O|

-------------------------------------------
Game end, winner is player O
Optimal player = O
RL Agent player = X
|- X -|
|O O O|
|X - X|

-------------------------------------------
Game end, winner is player O
Optimal player = O
RL Agent player = X
|O - X|
|X O X|
|O X O|

-------------------------------------------
Game end, winner is player X
Optimal player = X
RL Agent player = O
|O O X|
|X - X|
|O - X|

-------------------------------------------
Game end, winner is player None
Optimal player = O
RL Agent player = X
|X O X|
|X O O|
|O X X|

--------------------------------------

.